In [4]:
%pip install ollama pypdf

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [13]:
!pip install "langchain-chroma>=0.1.2"

^C


  Using cached langchain_chroma-0.2.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached chromadb-0.5.23-py3-none-any.whl.metadata (6.8 kB)
  Using cached fastapi-0.115.7-py3-none-any.whl.metadata (27 kB)
  Using cached numpy-1.26.4.tar.gz (15.8 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): still running...
  Preparing metadata (pyproject.toml): still running...
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached chroma_hnswlib-0.7.6-cp313-cp313-win_amd64.whl
  Using cached uvicorn-0.34.0-py3-none-any.whl.metadata (6.5 kB)
  Using 

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\User\\Documents\\LLM Research Paper\\venv\\Lib\\site-packages\\wrapt\\_wrappers.cp313-win_amd64.pyd'
Check the permissions.



In [18]:
%pip install -qU langchain-ollama

Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_community.utils.math import cosine_similarity
from langchain_ollama import OllamaEmbeddings
from langchain_ollama.llms import OllamaLLM
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
import ollama
from pypdf import PdfReader
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")



In [7]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma

llmgpt4o = ChatOpenAI(model="gpt-4o-mini")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [8]:
def getVectorStorage(collectionName, embeddingModel):
    return Chroma(
        collection_name= collectionName,
        embedding_function=embeddingModel,
        persist_directory='./chroma'
    )

In [9]:
def getTextData(pdfPath):
    reader = PdfReader(pdfPath)
    allText = ''
    for i in range(len(reader.pages)):
        read = reader.pages[i]
        allText += read.extract_text() + '\n'
    return allText

In [10]:
def getFromVector(query, vector_store):
    docs = vector_store.similarity_search(query, k=5) 
    return docs

In [11]:
def generateAnswer(query, vector_store, chat_llm):
    relevant_docs = getFromVector(query, vector_store)
    context = "\n".join([doc.page_content for doc in relevant_docs])

    prompt = f"Context: {context}\n\nQuestion: {query}\n\nAnswer:"
    response = chat_llm.invoke(prompt)
    return response

In [12]:
def PdfToChroma(text, vector_store):
    print("Storing in ",vector_store._collection_name)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    texts = text_splitter.split_text(text)
    
    docs = [Document(page_content=chunk) for chunk in texts]
    vector_store.add_documents(docs)
    print("PDF Stored Successfully")
    return "PDF Stored Successfully"

In [13]:
AllTestingDocuments = [
    "E-Assessment Tools for Programming Languages.pdf",
    "Analysis_Of_Medical_Data_Using_Data_Mini.pdf",
    "GuptaKB05.pdf"
]

In [14]:
StandardQuestions = [
    "What is the main research problem or question addressed in the paper?",
    "What methodology or approach has been used to solve the problem or answer the question?",
    "What are the key findings or results of the study?",
    "What are the limitations of the research mentioned in the paper?",
    "How does this research build upon or differ from previous studies in the field?",
    "What data or evidence has been used, and how was it collected or analyzed?",
    "What are the implications or practical applications of the findings?",
    "What assumptions are made in the study, and are they justified?",
    "Are there any unanswered questions or suggested areas for future research?",
    "How does the research contribute to its field, and why is it important?"
]

In [15]:
def getSummaryFromLLM(pdfPath, chat_llm):
    textData = getTextData(pdfPath)
    response = chat_llm.invoke(f" We need a brief summary in 2 pages of the text data provided below in : \n\n{textData}")
    return response

In [29]:
summary = getSummaryFromLLM("E-Assessment Tools for Programming Languages.pdf", llmgpt4o)

In [30]:
print(summary.content)

The paper discusses the need for automated assessment and feedback mechanisms in educational systems, particularly in programming courses within computer science. Continuous evaluation and feedback improve student learning and motivate ongoing effort, but traditional assessment methods become inadequate with scaling student populations. E-assessment tools could address these challenges, specifically designed for programming classes where tasks can be defined using formal language.

The authors review various grading techniques implemented by e-assessment tools, emphasizing the necessity for such tools to provide timely and constructive feedback. Tools that focus on programming help students practice more effectively, thereby enhancing their coding skills, while also reducing the workload for instructors by automating grading processes.

The review categorizes e-assessment tools based on assessment types (manual, automatic, semi-automatic), approaches (instructor-centered, student-cente

In [16]:
def getVectorEmbed(query):
    return embeddings.embed_query(query)

In [17]:
def getCosineSimilarity(summary_vector, standard_vector):
    return cosine_similarity([summary_vector], [standard_vector])

## Test

In [45]:
chat_llm = ChatOpenAI(model="gpt-4o-mini")

In [51]:
vector_store_raw = getVectorStorage("Raw_Analysis_Of_Medical_Data_Using_Data_Mini_GPT4oMINI", embeddings)
vector_store_summary = getVectorStorage("Summary_Analysis_Of_Medical_Data_Using_Data_Mini_GPT4oMINI", embeddings)

In [47]:
summary = getSummaryFromLLM("Analysis_Of_Medical_Data_Using_Data_Mini.pdf", chat_llm)

incorrect startxref pointer(1)
parsing for Object Streams


In [52]:
raw_document_text = getTextData("Analysis_Of_Medical_Data_Using_Data_Mini.pdf")

incorrect startxref pointer(1)
parsing for Object Streams


In [56]:
PdfToChroma(raw_document_text, vector_store_raw, "Raw_Analysis_Of_Medical_Data_Using_Data_Mini_GPT4oMINI")
PdfToChroma(summary.content, vector_store_summary, "Summary_Analysis_Of_Medical_Data_Using_Data_Mini_GPT4oMINI")

Storing in  Raw_Analysis_Of_Medical_Data_Using_Data_Mini_GPT4oMINI
PDF Stored Successfully
Storing in  Summary_Analysis_Of_Medical_Data_Using_Data_Mini_GPT4oMINI
PDF Stored Successfully


'PDF Stored Successfully'

## Mistral Model

In [18]:
import csv

In [23]:
def computeResult(model, vector_store_summary, vector_store_raw, chat_llm, pdfPath):
    raw_document_text = getTextData(pdfPath)
    summary = getSummaryFromLLM(pdfPath, chat_llm)

    PdfToChroma(raw_document_text, vector_store_raw)
    PdfToChroma(summary, vector_store_summary)
    res_data = [
        ["model", "vector_store", "type", "question", "answer"]
    ]

    stat_data = [
        ["question", "similarity", "model", "pdf"]
    ]

    for question in StandardQuestions:
        response_summary = generateAnswer(question, vector_store_summary, chat_llm)
        response_raw = generateAnswer(question, vector_store_raw, chat_llm)
        print("Question: ", question)
        similarity = getCosineSimilarity(getVectorEmbed(response_summary), getVectorEmbed(response_raw))
        print("Cosine Similarity: ", similarity)
        res_data.append([model, vector_store_summary._collection_name, "summary", question, response_summary])
        res_data.append([model, vector_store_raw._collection_name, "raw", question, response_raw])
        stat_data.append([question, similarity, model, pdfPath])


    with open(f'{model}-{''.join([x for x in pdfPath if x.isalnum()])}-results.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(res_data)

    with open(f'{model}-{''.join([x for x in pdfPath if x.isalnum()])}-stats.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(stat_data)

In [25]:
chat_llm = OllamaLLM(model="mistral:latest")
model = "Mistral"

for pdf in AllTestingDocuments:
    vector_store_raw = getVectorStorage(f"Raw_{''.join([x for x in pdf if x.isalnum()])}_{model}", embeddings)
    vector_store_summary = getVectorStorage(f"Summary_{''.join([x for x in pdf if x.isalnum()])}_{model}", embeddings)
    vector_store_raw.reset_collection()
    vector_store_summary.reset_collection()
    computeResult(model, vector_store_summary, vector_store_raw, chat_llm, pdf)

Storing in  Raw_EAssessmentToolsforProgrammingLanguagespdf_Mistral
PDF Stored Successfully
Storing in  Summary_EAssessmentToolsforProgrammingLanguagespdf_Mistral
PDF Stored Successfully


Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


Question:  What is the main research problem or question addressed in the paper?
Cosine Similarity:  [[0.7835297]]


Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


Question:  What methodology or approach has been used to solve the problem or answer the question?
Cosine Similarity:  [[0.71031378]]


...

Question:  What assumptions are made in the study, and are they justified?
Cosine Similarity:  [[0.69745201]]


Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


Question:  Are there any unanswered questions or suggested areas for future research?
Cosine Similarity:  [[0.67340905]]


Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


Question:  How does the research contribute to its field, and why is it important?
Cosine Similarity:  [[0.81078828]]


In [26]:
chat_llm = OllamaLLM(model="deepseek-r1:8b")
model = "Deepseek"

for pdf in AllTestingDocuments:
    vector_store_raw = getVectorStorage(f"Raw_{''.join([x for x in pdf if x.isalnum()])}_{model}", embeddings)
    vector_store_summary = getVectorStorage(f"Summary_{''.join([x for x in pdf if x.isalnum()])}_{model}", embeddings)
    vector_store_raw.reset_collection()
    vector_store_summary.reset_collection()
    computeResult(model, vector_store_summary, vector_store_raw, chat_llm, pdf)

Storing in  Raw_EAssessmentToolsforProgrammingLanguagespdf_Deepseek
PDF Stored Successfully
Storing in  Summary_EAssessmentToolsforProgrammingLanguagespdf_Deepseek
PDF Stored Successfully
Question:  What is the main research problem or question addressed in the paper?
Cosine Similarity:  [[0.78844307]]
Question:  What methodology or approach has been used to solve the problem or answer the question?
Cosine Similarity:  [[0.64525142]]
Question:  What are the key findings or results of the study?
Cosine Similarity:  [[0.73590628]]
Question:  What are the limitations of the research mentioned in the paper?
Cosine Similarity:  [[0.86145785]]
Question:  How does this research build upon or differ from previous studies in the field?
Cosine Similarity:  [[0.79896497]]
Question:  What data or evidence has been used, and how was it collected or analyzed?
Cosine Similarity:  [[0.71183108]]
Question:  What are the implications or practical applications of the findings?
Cosine Similarity:  [[0.860

incorrect startxref pointer(1)


Cosine Similarity:  [[0.76766025]]


parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams


Storing in  Raw_AnalysisOfMedicalDataUsingDataMinipdf_Deepseek
PDF Stored Successfully
Storing in  Summary_AnalysisOfMedicalDataUsingDataMinipdf_Deepseek
PDF Stored Successfully
Question:  What is the main research problem or question addressed in the paper?
Cosine Similarity:  [[0.82014362]]
Question:  What methodology or approach has been used to solve the problem or answer the question?
Cosine Similarity:  [[0.79846194]]
Question:  What are the key findings or results of the study?
Cosine Similarity:  [[0.83444306]]
Question:  What are the limitations of the research mentioned in the paper?
Cosine Similarity:  [[0.7661843]]
Question:  How does this research build upon or differ from previous studies in the field?
Cosine Similarity:  [[0.8993906]]
Question:  What data or evidence has been used, and how was it collected or analyzed?
Cosine Similarity:  [[0.6838798]]
Question:  What are the implications or practical applications of the findings?
Cosine Similarity:  [[0.75099797]]
Quest

In [27]:
chat_llm = OllamaLLM(model="llama3.2-vision:latest")
model = "LLAMA32"

for pdf in AllTestingDocuments:
    vector_store_raw = getVectorStorage(f"Raw_{''.join([x for x in pdf if x.isalnum()])}_{model}", embeddings)
    vector_store_summary = getVectorStorage(f"Summary_{''.join([x for x in pdf if x.isalnum()])}_{model}", embeddings)
    vector_store_raw.reset_collection()
    vector_store_summary.reset_collection()
    computeResult(model, vector_store_summary, vector_store_raw, chat_llm, pdf)

Storing in  Raw_EAssessmentToolsforProgrammingLanguagespdf_LLAMA32
PDF Stored Successfully
Storing in  Summary_EAssessmentToolsforProgrammingLanguagespdf_LLAMA32
PDF Stored Successfully


Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


Question:  What is the main research problem or question addressed in the paper?
Cosine Similarity:  [[0.77491212]]


Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


Question:  What methodology or approach has been used to solve the problem or answer the question?
Cosine Similarity:  [[0.67288142]]


...

Question:  How does this research build upon or differ from previous studies in the field?
Cosine Similarity:  [[0.72904223]]


Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


Question:  What data or evidence has been used, and how was it collected or analyzed?
Cosine Similarity:  [[0.77409706]]


Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


KeyboardInterrupt: 

In [30]:
chat_llm = OllamaLLM(model="mistral-nemo:latest")
model = "MistralNemo"

for pdf in AllTestingDocuments:
    vector_store_raw = getVectorStorage(f"Raw_{''.join([x for x in pdf if x.isalnum()])}_{model}", embeddings)
    vector_store_summary = getVectorStorage(f"Summary_{''.join([x for x in pdf if x.isalnum()])}_{model}", embeddings)
    vector_store_raw.reset_collection()
    vector_store_summary.reset_collection()
    computeResult(model, vector_store_summary, vector_store_raw, chat_llm, pdf)

Storing in  Raw_EAssessmentToolsforProgrammingLanguagespdf_MistralNemo
PDF Stored Successfully
Storing in  Summary_EAssessmentToolsforProgrammingLanguagespdf_MistralNemo
PDF Stored Successfully


Number of requested results 5 is greater than number of elements in index 2, updating n_results = 2


Question:  What is the main research problem or question addressed in the paper?
Cosine Similarity:  [[0.64473662]]


Number of requested results 5 is greater than number of elements in index 2, updating n_results = 2


Question:  What methodology or approach has been used to solve the problem or answer the question?
Cosine Similarity:  [[0.79860716]]


...

Question:  What assumptions are made in the study, and are they justified?
Cosine Similarity:  [[0.85041072]]


Number of requested results 5 is greater than number of elements in index 2, updating n_results = 2


Question:  Are there any unanswered questions or suggested areas for future research?
Cosine Similarity:  [[0.76282194]]


Number of requested results 5 is greater than number of elements in index 2, updating n_results = 2


Question:  How does the research contribute to its field, and why is it important?
Cosine Similarity:  [[0.87582577]]
Storing in  Raw_GuptaKB05pdf_MistralNemo
PDF Stored Successfully
Storing in  Summary_GuptaKB05pdf_MistralNemo
PDF Stored Successfully
Question:  What is the main research problem or question addressed in the paper?
Cosine Similarity:  [[0.79413961]]
Question:  What methodology or approach has been used to solve the problem or answer the question?
Cosine Similarity:  [[0.74914178]]
Question:  What are the key findings or results of the study?
Cosine Similarity:  [[0.77748904]]
Question:  What are the limitations of the research mentioned in the paper?
Cosine Similarity:  [[0.84886871]]
Question:  How does this research build upon or differ from previous studies in the field?
Cosine Similarity:  [[0.89082594]]
Question:  What data or evidence has been used, and how was it collected or analyzed?
Cosine Similarity:  [[0.5147976]]
Question:  What are the implications or prac